<a href="https://colab.research.google.com/github/ccwu0918/book-sqlfifty/blob/main/Sqlite_Flask_for_Food_Bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

## 為什麼要用資料庫？
- 可以隨心所欲的做資料的存取
- 當資料龐大時，處理效率比直接對檔案IO快上許多
- 方便跟各種服務系統串接

## 資料庫種類
- 關聯式資料庫：資料欄位須先定義清楚、穩定
- 非關聯式資料庫：資料擁有彈性結構、速度較快

## 常見的關聯式資料庫種類
- MySQL：免費、開源性質、常應用於Web，較適合中小型企業事務
- PostgreSQL：免費、開源、以商業應用導向為主
- MSSQL：Microsoft SQL Server，only on Windows，花錢就能買的完整方案。
- Oracle：公部門常用的資料庫系統，安全、效能、功能都極為齊全。
- SQLite：基於文件的資料庫，不需要架設伺服器，就能處理各種類數據。

## 常見的非關聯式資料庫種類
- MongoDB
- Couchbase
- Cassandra
- Redis
- memcache
- HBase
- Neo4J

![](http://image.slidesharecdn.com/mongodbrdbmsmigrationwebinar-140202060406-phpapp02/95/migrating-from-relational-databases-to-mongodb-10-638.jpg?cb=1391321199)

## Static
- 新建資料夾 `static`
- 在 `static` 資料夾中，新增檔案 `style.css`

## instance
- 新建資料夾 `instance`
- 在 `instance` 資料夾中，建立資料庫檔案 `food_bank.db`

## Template
- 新建資料夾 `templates`
- 在 `templates` 資料夾中，新增檔案 `food_bank.html`
- 在 `templates` 資料夾中，新增檔案 `view_food_bank.html`
- 在 `templates` 資料夾中，新增檔案 `edit_food_bank.html`



In [1]:
!mkdir ./static
!mkdir ./instance
!mkdir ./templates

## SQLite

``` python
import sqlite3
conn = sqlite3.connect('資料庫名稱.db')
sql = "......"
cursor = conn.execute(sql_ins)
cursor = conn.commit() ###  會修改到資料的必須要多寫此行
conn.close()
```

### 建立資料庫/資料表
- 建立資料庫(Database/DB)：`資料庫.db`
- 建立資料表(Table)：

SQLite 型別
- INTEGER：整數
- REAL：浮點數
- TEXT：文字
- BLOB：原始資料

``` mysql
CREATE TABLE `資料表` (
	`欄位1`	型別,
	`欄位2`	型別,
	`.....
)
```

In [2]:
!pip install SQLAlchemy==1.4.46

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10


In [ ]:
# !git clone https://github.com/ccwu0918/book-sqlfifty

In [22]:
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db') # 如果資料庫不存在，會自動幫你建立
sql_create_table = """
CREATE TABLE IF NOT EXISTS food_lists (
	`id` INTEGER PRIMARY KEY,
	`store` TEXT,
	`store_name` TEXT,
	`item_name` TEXT,
	`number` INTEGER,
	`price` INTEGER,
	`availability` TEXT,
	`expired_date` DATE,
	`booking_date` DATE
);
"""
cursor = conn.execute(sql_create_table)
conn.close()

## CRUD

### Create 新增資料
``` mysql
INSERT INTO `資料表` (`欄位1`, `欄位2`, ...)
VALUES (值1, 值2, )
```


### 一次新增多筆資料 (INSERT INTO SELECT)


語法：

``` mysql
INSERT INTO `資料表` (`欄位1`, `欄位2`, ...)
VALUES (value1_1, value1_2, ...),
    (value2_1, value2_2, ...),
    (value3_1, value3_2, ...),
    ... ;
或利用子查詢，從其它的資料表中取得資料來作一次多筆新增：

INSERT INTO table_name (column1, column2, column3,...)
SELECT othercolumn1, othercolumn2, othercolumn3,...
FROM othertable_name;
```

In [23]:
#@title 新增一筆訂房記錄
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db')

sql_ins = """
    INSERT INTO food_lists (store, store_name, item_name, number, price, availability, expired_date)
             VALUES ('7-11', '7-11金門門市', '經典總匯三明治', 3, 39, 'available', '2023/06/05');
"""
cursor = conn.execute(sql_ins)
cursor = conn.commit()
conn.close()

### Read 讀取資料
``` mysql
SELECT `欄位1`, `欄位2`, ...
FROM `資料表`
WHERE 條件1 and/or 條件2 ....
ORDER BY 排序方式1, 排序方式2
LIMIT 筆數限制
```
- 顯示欄位：
    - 全選所有資料表內的欄位 `SELECT * `
    - 可自訂新欄位(運算符號和Python相同) `SELECT 欄位1, (欄位1-欄位2)*100/欄位2`
    - 可自訂順序(不需要跟資料表順序一致)
- 條件：
    - 數值比較符號大致上和Python相同，
        - 比較兩邊相等：`=`
        - 沒有整除、次方運算子(要用[函式](https://www.w3schools.com/sql/sql_ref_mysql.asp))
- 排序方式：
    - DESC 倒序(大->小)排序
    - ASC 正序(小->大)排序(可省略不寫)

In [28]:
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db')

# SELECT `stock_id`,`date`,`open`,`close`,`high`,`low`,`volume`, `volume`*1000
# FROM `stock_date`
# WHERE `date` = '2020-03-03' or `volume` >= 40000
# ORDER BY `stock_id` DESC, `date`

sql = """
    SELECT * FROM food_lists ORDER BY id, expired_date DESC;
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

### Update 更新資料
``` mysql
UPDATE `資料表`
SET `欄位1`=值1, `欄位2`=值2, ...
WHERE 條件1 and/or 條件2 ....
```

In [25]:
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db')

id = 1
store = "全家"
store_name = "全家_金大門市"
item_name = "經典總匯三明治"
number = 2
price = 29
expired_date = "2023/06/06"

sql_upd = f"UPDATE food_lists SET store='{store}', store_name='{store_name}',item_name='{item_name}',number={number},price={price},expired_date='{expired_date}' WHERE id={id};"

cursor = conn.execute(sql_upd)
cursor = conn.commit()
conn.close()

### Delete 刪除資料
``` mysql
DELETE
FROM `資料表`
WHERE 條件1 and/or 條件2 ....
```

In [27]:
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db')

id = 1

# DELETE
# FROM `stock_date`
# WHERE `stock_id` = 2330 and `date` = '2020-03-03'

sql_del = f"DELETE FROM food_lists WHERE id={id};"

cursor = conn.execute(sql_del)
cursor = conn.commit()
conn.close()

### 處理重複資料
- 新增資料的時候先檢查資料是否存在：`SELECT`
- 設計資料庫：
    - 新增一欄位讓所有資料各擁有一唯一代碼，
        - 刪除時留下最小或最大的代碼：先用`SELECT`，再用`DELETE`
    - 建立唯一值組合：`UNIQUE INDEX`
        - 要先思考哪些欄位組合起來是唯一值

In [58]:
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db')
sql_create_ui = """
    CREATE UNIQUE INDEX `id_date` ON `food_lists`(`store_name`, `item_name`, `expired_date`)
"""
cursor = conn.execute(sql_create_ui)
cursor = conn.commit()
conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db')
sql_ins = """
    INSERT INTO food_lists (store, store_name, item_name, number, price, availability, expired_date)
             VALUES ('7-11', '7-11_金大門市', '經典總匯三明治', 3, 29, 'available', '2023/06/05'),
                 ('7-11', '7-11_下埔下門市', '經典三明治', 3, 25, 'available', '2023/06/05'),
                 ('全家', '全家_金大門市', '總匯三明治', 3, 29, 'available', '2023/06/05'),
                 ('全家', '全家_金寧門市', '經典三明治', 3, 25, 'available', '2023/06/05'),
                 ('全家', '全家_金湖門市', '雞肉三明治', 3, 30, 'available', '2023/06/05');
"""
cursor = conn.execute(sql_ins)
cursor = conn.commit()

In [ ]:
# sql_ins2 = """
#     INSERT INTO rooms (room_number, room_type, price, availability, booking_date) VALUES (101, '雙人房', 2000, 'unavailable', '2023/08/08');
# """
# cursor = conn.execute(sql_ins2)
# cursor = conn.commit()
# conn.close()

In [30]:
import sqlite3
conn = sqlite3.connect('./instance/food_bank.db')
sql = """
    SELECT *
    FROM `food_lists`
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

(1, '7-11', '7-11_金大門市', '經典總匯三明治', 3, 29, 'available', '2023/06/05', None)
(2, '7-11', '7-11_下埔下門市', '經典三明治', 3, 25, 'available', '2023/06/05', None)
(3, '全家', '全家_金大門市', '總匯三明治', 3, 29, 'available', '2023/06/05', None)
(4, '全家', '全家_金寧門市', '經典三明治', 3, 25, 'available', '2023/06/05', None)
(5, '全家', '全家_金湖門市', '雞肉三明治', 3, 30, 'available', '2023/06/05', None)


## MySQL

``` python
import pymysql
conn = pymysql.connect(host,account,passwd,dateabase)
cursor = conn.cursor()
sql = "......"
cursor.execute(sql)
cursor.commit()
conn.close()
```

### 環境設定

### [IMDB 資料庫](https://relational.fit.cvut.cz/dataset/IMDb)
![](https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg)


## INNER JOIN
- 一對多關係
    - 電影有很多演員
- 資料單一地方管理
    - `stock_list`：股票代碼、股票名稱
    - `stock_date`：股票代碼、日期、各交易原始資訊
    - 若股票名稱改名，改 `stock_list` 就好

``` mysql
SELECT `資料表n`.`欄位1`, `資料表n`.`欄位2`, ...
FROM `資料表1`
INNER JOIN `資料表2` ON 關聯條件1 and/or 條件2
WHERE 條件1 and/or 條件2 ....
ORDER BY 排序方式1, 排序方式2
LIMIT 筆數限制
```

## 學習資源
- [w3school](https://www.w3schools.com/sql/default.asp)
- [深入淺出 SQL](https://www.tenlong.com.tw/products/9789866840166)

# Flask

## 為什麼要學架網站？
- 讓任何人不限裝置/平台/系統，只要能上網，都能很方便且易使用
- 網路爬蟲抓的就是網站，所以理解網站怎麼架設，對於增加爬蟲經驗會很有幫助

## 為什麼要用 Flask ？
- 容易上手
- 輕量化

## 環境設定

In [4]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install flask==0.12.2

In [5]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=45d2e0e905114cadde66d3de32486d48eed7288e4b9bfd60e66d7e729cbb09ad
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


## 01.註冊 ngrok 帳號



[ngrok註冊網址](https://dashboard.ngrok.com/signup)

## 02. 建立並取得 authtoken


In [31]:
!ngrok authtoken '29Yeppg9EErt12XNjS2DlOVpMQv_4GHpNG5xMiBCPHwm2TzmD'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# !ngrok config add-authtoken 29Yeppg9EErt12XNjS2DlOVpMQv_4GHpNG5xMiBCPHwm2TzmD

## [Bootstrap](https://getbootstrap.com/)
- 包含 HTML、CSS及JavaScript的前端框架，有許多已經設定好的元素，方便開發

## [DataTables](https://datatables.net/)
- jQuery 套件，讓表格可以有更多功能操作
    - jQuery：JavaScript函式庫，簡化 HTML 與 JavaScript 之間的操作
- 此套件會把所有結果按照套件設定重新排序，亦即不會依照原先資料的排序

![](https://simpliv.files.wordpress.com/2018/04/learn-the-foundations-of-html-css-javascript-from-experts12.gif?w=640)

In [7]:
!pip install flask_bootstrap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask_bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460122 sha256=fb09e88f694f24cce6e6401dc6500feb9aab75528bff03c6295f427fa4122b95
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3925 sha256=e83e70a244e776444abdaa71ddd708374be71c30afb3f19ba2db60f4e43c1340
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d501a354366d180afc9f8bb2d333
Successfully built flask_bootstrap visitor


In [8]:
!pip install flask_sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/




---



# 網頁樣式設定

In [32]:
%%writefile ./static/style.css

body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
}

h1, h2 {
    text-align: center;
}

form {
    display: flex;
    flex-direction: column;
    align-items: center;
}

table {
    border-collapse: collapse;
    width: 100%;
    margin-top: 20px;
}

th, td {
    text-align: left;
    padding: 8px;
}

label {
    font-weight: bold;
}

a {
    text-decoration: none;
    color: #4CAF50;
}

th {
    background-color: #008080;
    color: white;
}

tr:nth-child(even) {
    background-color: #f2f2f2;
}

input[type="text"], input[type="date"] {
    padding: 5px;
    margin-bottom: 10px;
    border-radius: 5px;
    border: none;
}

input[type="submit"] {
    background-color: #008080;
    color: white;
    padding: 10px;
    border: none;
    border-radius: 5px;
    cursor: pointer;
    align-items: center;
}

input[type="submit"]:hover {
    background-color: #3e8e41;
}

Overwriting ./static/style.css




---



# 進階版網頁

In [63]:
%%writefile ./templates/food_bank.html
<head>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/dataTables.bootstrap4.min.css">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <script type="text/javascript" src="https://code.jquery.com/jquery-3.3.1.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/dataTables.bootstrap4.min.js"></script>
</head>
{% extends "bootstrap/base.html" %}

{% block title %}View Bookings{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
            data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">導覽</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">新增記錄</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">首頁</a></li>
                <li><a href="/setup">建立資料表</a></li>
                <li><a href="/insert_foods">新增範例資料</a></li>
                <li><a href="/view_foods">查詢記錄</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    <div class="page-header">
    <h1>新增即期商品</h1>
    <form method="POST">
        <label for="store">便利商店:</label>
        <input type="text" id="store" name="store" required><br>
        <label for="store_name">商店名稱:</label>
        <input type="text" id="store_name" name="store_name" required><br>
        <label for="item_name">商品名稱:</label>
        <input type="text" id="item_name" name="item_name" required><br>
        <label for="number">數量:</label>
        <input type="text" id="number" name="number" required><br>
        <label for="price">售價:</label>
        <input type="text" id="price" name="price" required><br>
        <label for="expired_date">有效期限:</label>
        <input type="date" id="expired_date" name="expired_date" required><br>
        <input type="submit" value="確定送出">
    </form>
    </div>
</div>
{% endblock %}

Overwriting ./templates/food_bank.html


In [66]:
%%writefile ./templates/view_foods.html
<head>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/dataTables.bootstrap4.min.css">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <script type="text/javascript" src="https://code.jquery.com/jquery-3.3.1.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/dataTables.bootstrap4.min.js"></script>
</head>
{% extends "bootstrap/base.html" %}

{% block title %}即期商品列表{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
            data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">導覽</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">即期商品記錄</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">首頁</a></li>
                <li><a href="/setup">建立資料表</a></li>
                <li><a href="/insert_foods">新增範例資料</a></li>
                <li><a href="/food_bank">新增記錄</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    <div class="page-header">
    <h1>即期商品記錄</h1>
    <table class="table" id="book_table">
        <thead>
            <tr>
                <th>便利商店</th>
                <th>商店名稱</th>
                <th>商品名稱</th>
                <th>數量</th>
                <th>售價</th>
                <th>Availability</th>
                <th>有效期限</th>
                <th>Action</th>
            </tr>
        </thead>
        <tbody>
            {% for food in foods %}
            <tr>
                <td>{{ food[1] }}</td>
                <td>{{ food[2] }}</td>
                <td>{{ food[3] }}</td>
                <td>{{ food[4] }}</td>
                <td>{{ food[5] }}</td>
                <td>{{ food[6] }}</td>
                <td>{{ food[7] }}</td>
                <td>
                    <a href="{{ url_for('edit_food', id=food[0]) }}">Edit</a> |
                    <a href="{{ url_for('delete_food', id=food[0]) }}">Delete</a>
                </td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <br>
    <form method="POST">
        <input type="submit" value="回首頁">
    </form>
    </div>
</div>
{% endblock %}
{% block scripts %}
<script type="text/javascript">
$(document).ready(function() {
    $('#book_table').DataTable();
} );
</script>
{% endblock %}

Overwriting ./templates/view_foods.html


In [65]:
%%writefile ./templates/edit_food.html
<head>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/dataTables.bootstrap4.min.css">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <script type="text/javascript" src="https://code.jquery.com/jquery-3.3.1.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/dataTables.bootstrap4.min.js"></script>
</head>
{% extends "bootstrap/base.html" %}

{% block title %}修改商品資訊{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
            data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">導覽</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">修改商品資訊</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">首頁</a></li>
                <li><a href="/setup">建立資料表</a></li>
                <li><a href="/insert_foods">新增範例資料</a></li>
                <li><a href="/view_foods">即期商品</a></li>
                <li><a href="/food_bank">新增記錄</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    <div class="page-header">
    <h1>修改商品資訊</h1>
    <form method="POST">
        {% for food in foods %}
        <label for="store">便利商店:</label>
        <input type="text" id="store" name="store" value="{{ food[1] }}" required><br>
        <label for="store_name">商店名稱:</label>
        <input type="text" id="store_name" name="store_name" value="{{ food[2] }}" required><br>
        <label for="item_name">商品名稱:</label>
        <input type="text" id="item_name" name="item_name" value="{{ food[3] }}" required><br>
        <label for="number">數量:</label>
        <input type="text" id="number" name="number" value="{{ food[4] }}" required><br>
        <label for="price">售價:</label>
        <input type="text" id="price" name="price" value="{{ food[5] }}" required><br>
        <label for="expired_date">有效期限:</label>
        <input type="date" id="expired_date" name="expired_date" value="{{ food[7] }}" required> ({{ food[7] }}) <br><br>
        {% endfor %}
        <input type="submit" value="確定修改">
    </form>
    </div>
</div>
{% endblock %}

Overwriting ./templates/edit_food.html




---



In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_sqlalchemy import SQLAlchemy

import sqlite3

app = Flask(__name__)

# 連接SQLite資料庫
# conn = sqlite3.connect('food_bank.db')

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///food_bank.db'
db = SQLAlchemy(app)
bootstrap = Bootstrap(app)
run_with_ngrok(app)


# 建立食物銀行資料表
@app.route('/setup')
def setup():
    sql = """
    CREATE TABLE IF NOT EXISTS food_lists (
        `id` INTEGER PRIMARY KEY,
        `store` TEXT,
        `store_name` TEXT,
        `item_name` TEXT,
        `number` INTEGER,
        `price` INTEGER,
        `availability` TEXT,
        `expired_date` DATE,
        `booking_date` DATE
    );
    """

    db.engine.execute(sql)
    # return "系統資料表建立成功！<a href=\"/\">回首頁</a>"
    return redirect(url_for('view_foods'))

@app.route('/', methods=['GET', 'POST'])

# 新增記錄功能
@app.route('/food_bank', methods=['GET', 'POST'])
def food_bank():
    if request.method == 'POST':
        store = request.form['store']
        store_name = request.form['store_name']
        item_name = request.form['item_name']
        number = request.form['number']
        price = request.form['price']
        availability = 'available'
        expired_date = request.form['expired_date']

        sql = f"""
            INSERT INTO food_lists (store, store_name, item_name, number, price, availability, expired_date)
                    VALUES ('{store}', '{store_name}', '{item_name}', {number}, {price}, '{availability}', '{expired_date}');
        """

        db.engine.execute(sql)
        return redirect(url_for('view_foods'))
    return render_template('food_bank.html')

# 新增多筆範例記錄
@app.route('/insert_foods')
def insert():
    try:                      # 使用 try，測試內容是否正確
        sql = """
            INSERT INTO food_lists (store, store_name, item_name, number, price, availability, expired_date)
                    VALUES ('7-11', '7-11_金大門市', '經典總匯三明治', 3, 29, 'available', '2023/06/05'),
                        ('7-11', '7-11_下埔下門市', '經典三明治', 3, 25, 'available', '2023/06/05'),
                        ('全家', '全家_金大門市', '總匯三明治', 3, 29, 'available', '2023/06/05'),
                        ('全家', '全家_金寧門市', '經典三明治', 3, 25, 'available', '2023/06/05'),
                        ('全家', '全家_金湖門市', '雞肉三明治', 3, 30, 'available', '2023/06/05');
        """

        db.engine.execute(sql)
        # return "多筆範例資料新增成功！<a href=\"/\">回首頁</a>"
    except:                   # 如果 try 的內容發生錯誤，就執行 except 裡的內容
        print("Error")
    return redirect(url_for('view_foods'))

# 查看即期商品紀錄
@app.route('/view_foods', methods=['GET', 'POST'])
def view_foods():
    if request.method == 'GET':
        sql = """
            SELECT * FROM food_lists ORDER BY id, expired_date DESC;
        """
        foods = db.engine.execute(sql)
        return render_template('view_foods.html', foods=foods)
    else:
        return redirect(url_for('food_bank'))

# 修改商品紀錄
@app.route('/edit_food/<int:id>', methods=['GET', 'POST'])
def edit_food(id):
    if request.method == 'POST':

        store = request.form['store']
        store_name = request.form['store_name']
        item_name = request.form['item_name']
        number = request.form['number']
        price = request.form['price']
        availability = 'unavailable'
        expired_date = request.form['expired_date']

        sql = f"UPDATE food_lists SET store='{store}', store_name='{store_name}',item_name='{item_name}',number={number},price={price},expired_date='{expired_date}' WHERE id={id};"

        db.engine.execute(sql)
        # return "資料修改成功！"
        return redirect(url_for('view_foods'))
    else:
        sql = f"SELECT * FROM food_lists WHERE id={id};"
        foods = db.engine.execute(sql)

        return render_template('edit_food.html', foods=foods)

# 刪除商品紀錄
@app.route('/delete_food/<int:id>')
def delete_food(id):
    sql = f"DELETE FROM food_lists WHERE id={id};"
    db.engine.execute(sql)
    # return "資料刪除成功！"
    return redirect(url_for('view_foods'))

# 使用參數刪除商品紀錄
@app.route("/delete_food")
def delete_booking_param():
    id = request.args.get('id')
    sql = f"DELETE FROM food_lists WHERE id={id};"
    db.engine.execute(sql)
    # return "資料刪除成功！"
    return redirect(url_for('view_foods'))

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cd79-34-23-255-244.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


#### [練習]
- 增加一個參數 expired_date
    - 如果是走 delete_food ， url 是 `/delete_food/1/XXX`
    - 如果是走 delete_food_param ， url 是 `/delete_food?id=1&booking_date=XXX`


## 學習資源
- [Flask Web 開發實戰](https://tw.pyladies.com/events/topic.html?id=35)
    - [Flask Web 開發:基於 Python 的 Web 應用開發實戰](https://www.tenlong.com.tw/products/9787115373991)
- 學習地圖
    - [前端](https://softnshare.com/webfrontendprogrammer/)
    - [後端](https://softnshare.com/backenddeveloper/)
        - VPS 主機：Linode, DigitalOcean, GCP, AWS, Azure
            - [鳥哥的 Linux 私房菜](http://linux.vbird.org/)
